In [19]:
import pandas as pd
import glob
import numpy as np
from math import modf, isnan
from pathlib import Path

In [20]:
def add_epsilon_dir(parsed_dir, geom):
    
    if not os.path.isdir(parsed_dir):
        raise Exception(f"Directory '{parsed_dir}' not found. Consider creating it and moving parsed files there.")
    
    CALIBRATION_PATH = "aux_data/epsilons.csv"
    eps_df = pd.read_csv(CALIBRATION_PATH, index_col=0)
    Path("./with_eps").mkdir(parents=True, exist_ok=True)
    for parsed_file in glob.iglob(f"{parsed_dir}/*.csv"):
        add_epsilon_file(parsed_file, geom, eps_df)

In [21]:
def add_epsilon_file(report_path, geom, eps_df):
    file_name = report_path.split('/')[1]
    df = pd.read_csv(report_path, index_col=0)
    
    df["Geometry"] = [geom for i in range(df.shape[0])]
    df["eps"] = df["E_tab"].apply(lambda x: add_epsilon_val(x, geom, eps_df))

    old_column_order = df.columns.tolist()
    first_cols = ["Pk", "Energy", "FWHM", "E_tab", "Area", "%err", "Ig", "eps", "Geometry"]
    new_cols = first_cols + list(set(old_column_order) - set(first_cols))
    df = df[new_cols]
    # that is wrong -- it's always 30 in the RPT files...
    df = df.drop(columns = ["Sample Geometry"])
    
    df.to_csv(f"with_eps/{file_name}")

In [25]:
def add_epsilon_val(num, geom, eps_df):
    if isnan(num):
        return float("NaN")
    else:
        dec_e, int_e = modf(num)
        int_e = int(int_e)
        
        curr_eps = float(eps_df.loc[int_e, f"eps_{geom}_mm"])
        eps = curr_eps + dec_e*(float(eps_df.loc[int_e + 1, f"eps_{geom}_mm"]) - curr_eps)
    return eps

In [26]:
add_epsilon_dir("with_Ig", 80)

['Pk', 'IT', 'Energy', 'Area', 'Bkgnd', 'FWHM', 'Channel', 'Left', 'PW', 'Cts/Sec', '%err', 'Fit', 'Report Generated On', 'Sample Identification', 'Sample Type', 'Sample Geometry', 'Peak Locate Threshold', 'Peak Locate Range (in channels)', 'Peak Area Range (in channels)', 'Identification Energy Tolerance', 'Sample Size', 'Sample Taken On', 'Acquisition Started', 'Live Time', 'Real Time', 'Dead Time', 'Peak Analysis Report                    26.11.2020  5', 'Peak Analysis Performed on', 'Peak Analysis From Channel', 'Peak Search Sensitivity', 'Max Iterations', 'Use Fixed FWHM', 'Peak Fit Engine Name', 'E_tab', 'Ig', 'eps']
